<img src="https://electronaut.info/wp-content/uploads/2017/05/Tubescape_jupyter2.jpg">

<h1><font color = b30000>TUBE DATA MATCHER for uTRACER FILES</font></h1><h5>Anode current for a sweep of grid voltages  when anode voltage = constant</h5>


# STEP 1 — Import and process tube data, build dataframe.

In [1]:
from electronaut_uTracer_Import import *
df.head()

Please enter the path to the folder containing the uTracer files: /Users/rrmc/github_rrMacKinnon/Electrona_uTracer/SampleTubeData
PLEASE NOTE: This script assumes all the files in the chosen directory are for the same type of tube!

What type of tubes are these? 5749

— — — — — 

Data from 49 tubes of type 5749 was successfully read.
A dataframe containing all 49 tubes was successfully created.


,tube_ID,tube_type,anode_voltage,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
0,1,5749,325,99.278,65.383,44.717,34.082,26.669,20.931,16.492,12.992,10.291,8.23,6.669,5.495,4.626
1,10,5749,325,84.662,55.752,37.5,27.666,20.873,15.84,12.088,9.304,7.261,5.807,4.747,3.945,3.344
2,11,5749,325,57.956,39.019,26.465,19.832,14.986,11.21,8.358,6.284,4.834,3.863,3.22,2.752,2.418
3,12,5749,325,85.883,52.717,33.817,24.6,18.367,13.795,10.481,8.071,6.345,5.144,4.268,3.592,3.043
4,13,5749,325,90.664,57.346,37.541,27.686,21.015,16.003,12.239,9.433,7.395,5.967,4.977,4.267,3.759


##### TODO:  INSERT PLOT OF ALL TUBES

# STEP 3: Show the statistics of the tube data set

In [2]:
df_stats = df.astype(float).describe()
df_stats.loc[:, 'tube_type'] = df_stats.loc[:, 'tube_type'].astype(int)
df_stats.loc[:, 'tube_type'] = df_stats.loc[:, 'tube_type'].astype(str)
df_stats

,tube_ID,tube_type,anode_voltage,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
count,49.000000,49,49.0,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,25.183673,5749,325.0,84.799673,54.311041,36.111408,26.890265,20.579020,15.856878,12.301735,9.640918,7.677776,6.246265,5.190184,4.378469,3.770286
std,14.554028,0,0.0,9.698785,6.042972,4.372425,3.511439,3.065207,2.746162,2.472266,2.223436,1.987546,1.758411,1.545969,1.361598,1.225021
min,1.000000,5749,325.0,55.209000,38.328000,25.882000,19.516000,14.487000,10.815000,8.044000,5.992000,4.557000,3.600000,2.975000,2.525000,2.180000
25%,13.000000,5749,325.0,80.152000,51.370000,33.824000,24.949000,18.770000,14.291000,10.876000,8.414000,6.599000,5.211000,4.268000,3.592000,3.043000
50%,25.000000,5749,325.0,86.052000,54.921000,36.069000,26.743000,20.632000,15.688000,12.030000,9.304000,7.318000,5.906000,4.889000,4.090000,3.487000
75%,37.000000,5749,325.0,90.054000,57.685000,38.138000,28.520000,21.656000,17.533000,14.101000,11.081000,8.787000,7.086000,5.857000,4.969000,4.293000
max,50.000000,5749,325.0,100.431000,65.603000,46.996000,36.666000,30.080000,25.292000,21.460000,18.384000,15.898000,13.867000,12.198000,10.760000,9.616000


##### TODO:  INSERT PLOT(S) OF STATISTICS 

# STEP 4: Calculate the differences between all tubes

In [3]:
def build_difference_df(df, df_stats):
       
    # Make a list of all the columns in the dataframe
    cols = list(df.columns)
    
    # Make a list of just the bias columns
    bias_list = []
    for i in cols:
        if i.startswith('Bias_') == True:
            bias_list.append(i)
    
    # BMake an empty list to hold the calculated values, to be used to build the difference dataframe
    diff_squared_list = []
    
    # Iterate through the index of tubes in the master dataframe to select a reference tube
    for each_ref_tube in df.index:
        
        # Get the next reference tube data, as a series
        ref_tube = df.iloc[each_ref_tube]
        ref_tube_ID = ref_tube.tube_ID

        # Iterate through the index of tubes in the master dataframe to select a match tube
        for each_match_tube in df.index:
            
            # Get the next tube data to be compared, as a series
            match_tube = df.iloc[each_match_tube]
            match_tube_ID = match_tube.tube_ID
            
            # Dictionary container to hold the difference variables for each tube
            temp_dict = {}
            
            # Compute the squares of the differences in each current measurement
            error_sum = 0
            for bias in enumerate(bias_list):
                mismatch = ((float(match_tube.loc[bias[1]]) - float(ref_tube.loc[bias[1]]))**2)
                temp_dict[bias[1]] = mismatch
                error_sum = error_sum + mismatch

            # Add other relevant key/value pairs to the dictionary  
            temp_dict['ref_tube_ID'] = ref_tube_ID
            temp_dict['match_tube_ID'] = match_tube_ID
            temp_dict['error_sum'] = error_sum

            # Append the tube's calculated values to the diff_squared_list
            diff_squared_list.append(temp_dict)
        
    df_dif = pd.DataFrame(diff_squared_list)
    return df_dif
    
    
df_dif = build_difference_df(df, df_stats)

In [5]:
df_dif.head()

,Bias_10,Bias_14,Bias_18,Bias_2,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50,Bias_6,error_sum,match_tube_ID,ref_tube_ID
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,1
1,52.085089,41.165056,33.593616,213.627456,25.918281,19.395216,13.601344,9.180900,5.870929,3.694084,2.402500,1.643524,92.756161,514.934156,10,1
2,333.135504,203.062500,136.492489,1707.507684,94.497841,66.161956,44.997264,29.778849,19.070689,11.895601,7.524049,4.875264,695.060496,3354.060186,11,1
3,118.810000,89.908324,68.923204,179.426025,50.922496,36.132121,24.216241,15.570916,9.523396,5.764801,3.621409,2.505889,160.427556,765.752378,12,1
4,51.494976,40.908816,31.967716,74.200996,24.285184,18.088009,12.666481,8.386816,5.121169,2.862864,1.507984,0.751689,64.593369,336.836069,13,1


In [15]:
import ipywidgets as wg
from IPython.display import display

# Make a list of all the tube_ID names, then build a select box with those values
tube_list = list(df.loc[:, 'tube_ID'])
tube_list.sort(key=int)
tube_ID_select_box = wg.Select(options=tube_list, value=tube_list[0], description='Reference Tube', disabled=False)

# Make a list of the tubes for the "Matched Tubes" selectMultiple box
matched_list = ['1', '33', '67', '22', '45']
matched_tube_box = wg.SelectMultiple(options=matched_list, description="Matched Tubes")

# def respond(sender):
    # pass the chosen Tube_ID to a function that looks up the resulting match list, then fill the match list box
    
    
display(tube_ID_select_box,matched_tube_box)
# tube_ID_select_box.observe(respond)

# STEP 5: Filter the results to find matches of any set size

In [ ]:
# Enter the tube to be matched and the size of the matched set
tube_to_match = 1

tube_set_size = 8

tube_to_match = str(tube_to_match)

# Sorting the dataframe to show the nearest matches to the chosen tube_to_match
df_dif[(df_dif.ref_tube_ID == tube_to_match)].sort_values('error_sum').head(tube_set_size)


### Make a function that receives tube_to_match and tube_set_size arguments, adds up the total of all the error_sum values per set for a set score, then sorts and displays all set scores.

### Consider adding a column called "available", and any matched sets can be removed from the remaining dataset by toggling the availability to False.

# Miscellaneous stuff

##### Apply a function to a column
df.Bias_2.max()

In [ ]:
# Applymap is useful for applying a function to all data
# For example, converting all data from floats to ints
# df.loc[2:7, 'Bias_2': 'Bias_50'].applymap(int)

In [ ]:
# List all available magics
% lsmagic

In [ ]:
# Display environmental variables
env_variables = %env
current_path = env_variables.get('PATH')
current_path

In [ ]:
mydict = % env
mydict.keys()

In [ ]:
% env

In [ ]:
# % time x = range(10000)
% timeit x = range(10000)
max(x)